# code based on open route services example

## please read: 
https://openrouteservice.org/2187-2/
https://saralgyaan.com/posts/set-passwords-and-secret-keys-in-environment-variables-maclinuxwindows-python-quicktip/
https://openrouteservice.org/dev/#/api-docs/optimization/post

In [1]:
import folium
import pandas as pd
import geopandas as gpd 
from openrouteservice import client
import json
import time
import glob
import rtree

In [2]:
import os

create_isochromes = False

# in .bashrc add export API_KEY="[api key here]"
# so do this:
# nano .bashrc
# export API_KEY="[api key here]"
# ctrl+X, Y

# load jupyter notebook with:
# env API_KEY=$API_KEY jupyter notebook

api_key = os.environ.get('API_KEY') # Provide your personal API key
# you need to make one on the openrouteservices website. 

print(api_key != None) # DO NOT PRINT YOUR API KEY, OR PUSH IT TO GITHUB!!!!

True


#### Part 1 - adapted from https://www.linkedin.com/pulse/isochrones-geopandas-paul-whiteside/

In [3]:
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

In [4]:
train_sf = gpd.read_file("../data/raw/shapefiles/Train_station_location/PTV_train/PTV_METRO_TRAIN_STATION.shp")
trainreg_sf = gpd.read_file("../data/raw/shapefiles/Train_station_location_reg/PTV_train_reg/PTV_REGIONAL_TRAIN_STATION.shp")

In [5]:
train_df = pd.DataFrame(train_sf)
train_df = train_df.drop(['STOP_NAME', 'TICKETZONE', 'ROUTEUSSP', 'geometry'], axis=1)
train_df.shape

(220, 3)

In [6]:
trainreg_df = pd.DataFrame(trainreg_sf)
trainreg_df = trainreg_df.drop(['STOP_NAME', 'geometry'], axis=1)
trainreg_df.shape

(110, 3)

In [7]:
train_jointdf = train_df.append(trainreg_df,ignore_index=True)
train_jointdf.shape

/tmp/ipykernel_42711/2273636930.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_jointdf = train_df.append(trainreg_df,ignore_index=True)


(330, 3)

In [8]:
l_processed = []
def get_isochrones():
    
    for STOP_ID, LATITUDE, LONGITUDE in train_jointdf.values:

        if not STOP_ID in l_processed:
            point = [LATITUDE, LONGITUDE]
            params_iso = {'profile': 'driving-car',
              'range': [600, 1200, 1800, 2400],  #10, 20, 30 mins
              'segments': 600,
              'attributes': ['total_pop'],  # Get population count for isochrones
              'locations':[point[::-1]]
              }

            try:
                clnt = client.Client(key=api_key)
                r = clnt.isochrones(**params_iso)

                for feature in r['features']:
                    feature['properties']['name'] = STOP_ID

                with open(f'../data/curated/isochrones/{STOP_ID}2.json', 'w') as f:
                    f.write(json.dumps(r))
                    l_processed.append(STOP_ID)
            except:
                print(f"Problem processing {STOP_ID}")

            
            time.sleep(2)

In [10]:
# Call function to find isochrones - not required to call again
if not create_isochromes: 
    get_isochrones()
    create_isochromes = True

/home/sesshah/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/sesshah/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/sesshah/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/sesshah/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


In [11]:
l_dfs = []

for file in glob.glob('../data/curated/isochrones/*'):
    with open(file) as json_file:
        data = json.load(json_file)

        gdf = gpd.GeoDataFrame.from_features(data)
        l_dfs.append(gdf)

In [12]:
gdf_isochrones = pd.concat(l_dfs)

In [13]:
gdf_isochrones.shape

(1648, 6)

In [14]:
gdf_isochrones.set_crs(epsg=7844, inplace=True)

matrix = gpd.sjoin(gdf_isochrones, vic_sf, how='inner', op='contains')\
    .loc[:,['SA2_NAME21', 'value', 'name']]\
    .set_index(['SA2_NAME21', 'value'])

matrix = matrix.reset_index()
matrix.columns = ['suburb', 'duration_mins', 'train_station_id']
matrix['duration_mins'] = matrix.duration_mins/60
matrix = matrix.groupby(['suburb', 'train_station_id']).duration_mins.min().to_frame().reset_index()

/home/sesshah/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [15]:
matrix

,suburb,train_station_id,duration_mins
0,Abbotsford,19835,20.0
1,Abbotsford,19837,30.0
2,Abbotsford,19838,30.0
3,Abbotsford,19839,30.0
4,Abbotsford,19840,30.0
...,...,...,...
39140,Yarraville,46468,30.0
39141,Yarraville,47647,40.0
39142,Yarraville,47648,30.0
39143,Yarraville,48804,40.0


Location of Suburb Centeroids within 20 Minute Drive of Train Stations "45793" and "15353"

In [16]:

IDs = ["45793", "15353"]
minutes = 20

# set up folium map
map = folium.Map(location=[-37.78, 145.29], zoom_start=9)

for STOP_ID in IDs:
    with open(f'../data/curated/isochrones/{STOP_ID}2.json', 'r') as f:
        geojson = json.loads(f.read())

    point = geojson['features'][0]['properties']['center'][::-1]

    folium.features.GeoJson(geojson).add_to(map)
    folium.map.Marker(point).add_to(map)

    suburbs = matrix.query("train_station_id == @STOP_ID and duration_mins <= @minutes").values[:,0]
    for s in suburbs:
        sub = vic_sf.loc[vic_sf['SA2_NAME21'] == s]
        name = sub['SA2_NAME21']
        pdgeo = sub['geometry'].to_crs('+proj=cea').centroid.to_crs(sub['geometry'].crs)
        folium.Circle([pdgeo.y, pdgeo.x], 250, fill=True).add_child(folium.Popup(name)).add_to(map)

map

#### Part 2

In [17]:
df = pd.read_csv("../data/raw/full_property_data.csv")

coords_cols = ["longitude", "latitude"]
houses_df = df[["index", *coords_cols]]
houses_df["location"] = houses_df[coords_cols].values.tolist()

houses_df.set_index("index", inplace=True)
houses_df.drop(coords_cols, axis=1, inplace=True)
min_houses = houses_df[0:3]
min_houses.head()

/tmp/ipykernel_42711/3610457353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_df["location"] = houses_df[coords_cols].values.tolist()
/tmp/ipykernel_42711/3610457353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_df.drop(coords_cols, axis=1, inplace=True)


,location
index,
https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767,"[144.9746821, -37.8650177]"
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303,"[144.9740049, -37.8141725]"
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455,"[144.9569041, -37.8301164]"


In [18]:
# set up the houses dictionary 
houses = min_houses.to_dict('index')
houses

{'https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767': {'location': [144.9746821,
   -37.8650177]},
 'https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303': {'location': [144.9740049,
   -37.8141725]},
 'https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455': {'location': [144.9569041,
   -37.8301164]}}

In [19]:
ors = client.Client(key=api_key)
# Set up folium map
map1 = folium.Map(location=[-37.86, 144.97], tiles="Stamen Terrain", zoom_start=10, preferCanvas=True)

# Request of isochrones with 15 minute on foot.
params_iso = {'profile': 'foot-walking',
              'range': [900],  # 900/60 = 15 minutes
              'attributes': ['total_pop']  # Get population count for isochrones
              }

for name, house in houses.items():
    print(name)
    params_iso['locations'] = [house['location']]  # Add apartment coords to request parameters
    house['iso'] = ors.isochrones(**params_iso)  # Perform isochrone request
    folium.features.GeoJson(house['iso']).add_to(map1)  # Add GeoJson to map

    folium.map.Marker(list(reversed(house['location'])),  # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                       icon_color='#cc0000',
                                       icon='home',
                                       prefix='fa',
                                       ),
                      popup=name,
                      ).add_to(map1)  # Add apartment locations to map

map1

https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455


In [20]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance',
             'limit': 3}

# POI categories according to
# https://giscience.github.io/openrouteservice/documentation/Places.html
categories_poi = {'bus_stop': [588],
                  'bus_station': [587],
                  'railway_station': [604],
                  'train_station': [610],
                  'tram_stop': [605]}

for name, house in houses.items():
    house['categories'] = dict()  # Store in pois dict for easier retrieval
    params_poi['geojson'] = house['iso']['features'][0]['geometry']
    print("\n{} apartment".format(name))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        house['categories'][typ] = dict()
        house['categories'][typ]['geojson'] = ors.places(**params_poi)['features']  # Actual POI request
        print(f"\t{typ}: {len(house['categories'][typ]['geojson'])}")


https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 0
	train_station: 0
	tram_stop: 2

https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 2
	train_station: 2
	tram_stop: 2

https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455 apartment
	bus_stop: 0
	bus_station: 0
	railway_station: 0
	train_station: 0
	tram_stop: 2


In [21]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
                'format_out': 'geojson',
                'geometry': 'true',
                'format': 'geojson',
                'instructions': 'false',
                "radiuses":[400]
                }

# Set up dict for font-awesome
style_dict = {'bus_stop': 'child',
              'train_station': 'shopping-cart',
              'tram_stop': 'scissors',
              'railway_station': 'child',
              'bus_station': 'scissors'
              }

# Store all routes from all apartments to POIs
for house in houses.values():
    for cat, pois in house['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']

            # Perform actual request
            params_route['coordinates'] = [house['location'],
                                           poi_coords
                                           ]
            json_route = ors.directions(**params_route)

            folium.features.GeoJson(json_route).add_to(map1)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                               )
                              ).add_to(map1)

            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration)  # Record durations of routes

map1

In [22]:
# Sum up the closest POIs to each apartment
for name, house in houses.items():
    house['shortest_sum'] = sum([min(cat['durations'] + [300]) for cat in house['categories'].values()])
    print(f"{name} apartment: {round(house['shortest_sum'] / 60, 1)} min"
          )

https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767 apartment: 25.0 min
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303 apartment: 24.9 min
https://www.domain.com.au/901-902-85-market-street-south-melbourne-vic-3205-14089455 apartment: 25.0 min


In [39]:
IDs = ["45793", "15353"]
minutes = 20

for STOP_ID in IDs:
    with open(f'../data/curated/isochrones/{STOP_ID}2.json', 'r') as f:
        geojson = json.loads(f.read())

    point = geojson['features'][0]['properties']['center'][::-1]
    for i in range(0,4):
        print((geojson['features'][i]['geometry']['coordinates'][0])) # this is a set of coords we can convert to poly
    
    
    # here, need to iterate through the properties and assign the closest isochrome based on the polygon

#    suburbs = matrix.query("train_station_id == @STOP_ID and duration_mins <= @minutes").values[:,0]
#    for s in suburbs:
#        sub = vic_sf.loc[vic_sf['SA2_NAME21'] == s]
#        name = sub['SA2_NAME21']
#        pdgeo = sub['geometry'].to_crs('+proj=cea').centroid.to_crs(sub['geometry'].crs)
#        folium.Circle([pdgeo.y, pdgeo.x], 250, fill=True).add_child(folium.Popup(name)).add_to(map)


[[145.153576, -38.089978], [145.153946, -38.090842], [145.155068, -38.093254], [145.15662, -38.093252], [145.16448, -38.08994], [145.17427, -38.094859], [145.175096, -38.096959], [145.183507, -38.102833], [145.186142, -38.103944], [145.193279, -38.111506], [145.194757, -38.113387], [145.198999, -38.119583], [145.20056, -38.121727], [145.20409, -38.122437], [145.215558, -38.1244], [145.21673, -38.126925], [145.213276, -38.137056], [145.209326, -38.140762], [145.209707, -38.142133], [145.210928, -38.143986], [145.218352, -38.14676], [145.224512, -38.154238], [145.223204, -38.159719], [145.223127, -38.160015], [145.220413, -38.166942], [145.219626, -38.169604], [145.219613, -38.169678], [145.219341, -38.171984], [145.222894, -38.172565], [145.224451, -38.171057], [145.229714, -38.160419], [145.230342, -38.159898], [145.231114, -38.158865], [145.238946, -38.151194], [145.245477, -38.14961], [145.247035, -38.149007], [145.247081, -38.148928], [145.247338, -38.147822], [145.247405, -38.14745

In [35]:
from shapely import geometry

# this function converts two floats to a point that is recognised by Shapely
def convert_to_point(row):
    x = float(row["latitude"])
    y = float(row["longitude"])
    return Point(y, x)

# this function loops through the victorian suburbs and tries to locate which zone a 
# property is in, based on its coordinates
def find_zone(point, geom):
    poly = geometry.Polygon([[p.x, p.y] for p in geom])
    if point.within(geom):
        return True
    return None